In [2]:
import numpy as np
import cv2

In [3]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [27]:
import imageio.v3 as iio
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    # ------------TODO --------------
    fx = K[0,0]
    fy = K[1,1]
    cx = K[0,2]
    cy = K[1,2]

    pc = []
    height, width = depth.shape
    mask = (seg == 255)

    z = depth[mask]
    y, x = np.nonzero(mask) #获取前景索引
    x = (x - cx) * z / fx
    y = (y - cy) * z / fy

    # 构造点云
    pc = np.column_stack((x, y, z))
    pc = np.array(pc)
    
    return pc


partial_pc = depth2pc(dpt, seg, K)

#可视化检验结果
#pcd_o3d = o3d.geometry.PointCloud() 
#pcd_o3d.points = o3d.utility.Vector3dVector(partial_pc) 
#o3d.visualization.draw_geometries([pcd_o3d])

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [33]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------
def nearest_point(p): #找出full_pc_sampled中离p最近的点
    distances = np.linalg.norm(p - full_pc_sampled, axis = 1)
    dis_min = np.argmin(distances)
    nearest = full_pc_sampled[dis_min]
    return nearest

def cal_nearest_dis(p): #离p最近的点距离p的distance
    nearest = nearest_point(p)
    return np.sqrt(np.sum(np.square(p-nearest)))

vectorized_nearest_dis = np.vectorize(cal_nearest_dis)
nearest_dis = np.apply_along_axis(cal_nearest_dis, axis=1, arr=partial_pc_sampled)

one_way_CD = np.mean(nearest_dis)


print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', [one_way_CD])

one way chamfer distance:  0.009902685909389496
